In [2]:
pip install kivy

                                              0.0/4.0 MB ? eta -:--:--
     --                                       0.2/4.0 MB 4.1 MB/s eta 0:00:01
     -----                                    0.6/4.0 MB 6.0 MB/s eta 0:00:01
     --------                                 0.8/4.0 MB 6.0 MB/s eta 0:00:01
     -----------                              1.1/4.0 MB 5.9 MB/s eta 0:00:01
     --------------                           1.4/4.0 MB 6.1 MB/s eta 0:00:01
     ----------------                         1.7/4.0 MB 6.0 MB/s eta 0:00:01
     --------------------                     2.0/4.0 MB 6.2 MB/s eta 0:00:01
     -----------------------                  2.3/4.0 MB 6.2 MB/s eta 0:00:01
     -----------------------                  2.4/4.0 MB 6.1 MB/s eta 0:00:01
     ------------------------                 2.5/4.0 MB 5.3 MB/s eta 0:00:01
     ----------------------------             2.9/4.0 MB 5.5 MB/s eta 0:00:01
     ----------------------------             2.9/4.0 MB 5.5 MB/s eta 0


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: C:\Users\wizard\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


In [1]:
import cv2
import mediapipe as mp
from kivy.app import App
from kivy.uix.boxlayout import BoxLayout
from kivy.uix.image import Image
from kivy.uix.button import Button
from kivy.graphics.texture import Texture
from kivy.graphics import Line
from kivy.clock import Clock

[INFO   ] [Logger      ] Record log in C:\Users\wizard\.kivy\logs\kivy_23-08-18_1.txt
[INFO   ] [deps        ] Successfully imported "kivy_deps.angle" 0.3.3
[INFO   ] [deps        ] Successfully imported "kivy_deps.glew" 0.3.1
[INFO   ] [deps        ] Successfully imported "kivy_deps.sdl2" 0.6.0
[INFO   ] [Kivy        ] v2.2.1
[INFO   ] [Kivy        ] Installed at "C:\Users\wizard\AppData\Local\Programs\Python\Python310\lib\site-packages\kivy\__init__.py"
[INFO   ] [Python      ] v3.10.0 (tags/v3.10.0:b494f59, Oct  4 2021, 19:00:18) [MSC v.1929 64 bit (AMD64)]
[INFO   ] [Python      ] Interpreter at "C:\Users\wizard\AppData\Local\Programs\Python\Python310\python.exe"
[INFO   ] [Logger      ] Purge log fired. Processing...
[INFO   ] [Logger      ] Purge finished!
[INFO   ] [Factory     ] 190 symbols loaded
[INFO   ] [Image       ] Providers: img_tex, img_dds, img_sdl2, img_pil (img_ffpyplayer ignored)
[INFO   ] [Text        ] Provider: sdl2


In [3]:


class CameraApp(App):
    def build(self):
        self.layout = BoxLayout(orientation='vertical')
        
        # Create a frame for displaying video
        self.video_frame = Image(size_hint=(1, 0.8))
        self.layout.add_widget(self.video_frame)

        # Create start and stop buttons
        self.start_button = Button(text='Start', size_hint=(0.2, 0.1))
        self.stop_button = Button(text='Stop', disabled=True, size_hint=(0.2, 0.1))
        self.start_button.bind(on_press=self.start_capture)
        self.stop_button.bind(on_press=self.stop_capture)
        self.layout.add_widget(self.start_button)
        self.layout.add_widget(self.stop_button)

        # Create an instance of the hand detection module
        self.mp_hands = mp.solutions.hands
        self.hands = self.mp_hands.Hands(
            static_image_mode=False, max_num_hands=2,
            min_detection_confidence=0.8, min_tracking_confidence=0.6
        )
        self.mp_drawing = mp.solutions.drawing_utils


        self.capture = None
        self.is_capturing = False
        self.show_cam = True
        self.frames = []
        print('cam is ready')
        self.capture = cv2.VideoCapture(0)  # Open the camera
        print('cam on')

        self.roi = (40, 30, 400, 400)  # Region of Interest (x, y, width, height)

        # Display initial camera feed
        self.update(0)

        Clock.schedule_interval(self.update, 1.0 / 30.0)  # Schedule video update

        return self.layout

    def start_capture(self, instance):
        self.capture = cv2.VideoCapture(0)
        self.is_capturing = True
        self.start_button.disabled = True
        self.stop_button.disabled = False
        Clock.schedule_interval(self.update, 1.0 / 30.0)

    def stop_capture(self, instance):
        self.is_capturing = False
        self.capture.release()  # Release the camera
        self.start_button.disabled = False
        self.stop_button.disabled = True
        Clock.unschedule(self.update)

        # Save the captured frames as a video file
        if self.frames:
            self.save_video(self.frames)

    def update(self, dt):
        if self.is_capturing:
            ret, frame = self.capture.read()  # Read a frame from the camera
            if ret:
                # Flip the frame vertically to correct orientation
                frame = cv2.flip(frame, 0)
                
                # Draw borders around ROI
                if self.roi:
                    x, y, w, h = self.roi
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

                # Convert the frame from BGR to RGB
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

                # Update the Kivy Image widget
                self.video_frame.texture = self.convert_to_kivy_texture(frame_rgb)

                # Store the frame for saving as a video (in ROI only)
                if self.roi:
                    self.frames.append(frame_rgb[y:y+h, x:x+w])

                    self.detect_and_draw_hands(frame_rgb)
        elif self.show_cam:
            ret, frame = self.capture.read()  # Read a frame from the camera
            if ret:
                # Flip the frame vertically to correct orientation
                frame = cv2.flip(frame, 0)
                
                # Draw borders around ROI
                if self.roi:
                    x, y, w, h = self.roi
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

                # Convert the frame from BGR to RGB
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

                # Update the Kivy Image widget
                self.video_frame.texture = self.convert_to_kivy_texture(frame_rgb)

                self.detect_and_draw_hands(frame_rgb)
    
    def detect_and_draw_hands(self, frame_rgb):
        results = self.hands.process(frame_rgb)
        if results.multi_hand_landmarks:
            for landmarks in results.multi_hand_landmarks:
                self.mp_drawing.draw_landmarks(frame_rgb, landmarks, self.mp_hands.HAND_CONNECTIONS)
        
        # Update the Kivy Image widget
        self.video_frame.texture = self.convert_to_kivy_texture(frame_rgb)

    def convert_to_kivy_texture(self, cv2_frame):
        texture = Texture.create(size=(cv2_frame.shape[1], cv2_frame.shape[0]), colorfmt='rgb')
        texture.blit_buffer(cv2_frame.tostring(), colorfmt='rgb', bufferfmt='ubyte')
        return texture

    def save_video(self, frames):
        out = cv2.VideoWriter('captured_video.avi', cv2.VideoWriter_fourcc(*'XVID'), 30, frames[0].shape[:2][::-1])

        for frame in frames:

            # Flip the frame vertically to correct orientation
            frame = cv2.flip(frame, 0)
            
            # Convert frame from BGR to RGB before saving
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            
            # Remove the green border (ROI) from the frame
            if self.roi:
                x, y, w, h = self.roi
                green_border_size = 0
                roi_frame = rgb_frame[y:y+h-green_border_size, x:x+w-green_border_size]  # Extract ROI region
                
                # Resize ROI frame to match the dimensions of the saved frames
                target_size = (frames[0].shape[1], frames[0].shape[0])
                roi_frame_resized = cv2.resize(roi_frame, target_size, interpolation=cv2.INTER_LINEAR)
                
                out.write(roi_frame_resized)
            else:
                out.write(rgb_frame)  # Save the entire frame

        out.release()




if __name__ == '__main__':
    CameraApp().run()


OSError: source code not available